In [ ]:
import numpy as np
import healpy as hp

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time

from astroplan import Observer, FixedTarget
from astroplan.plots import plot_airmass

import skyproj

import matplotlib.pyplot as plt
#%matplotlib widget

# Candidate Fields

Characteristics of a set of candidate fields

In [ ]:
targets = [
    FixedTarget(SkyCoord(59.1004, -48.73, unit='deg', frame='icrs'), name='EDFS ComCam'),
    FixedTarget(SkyCoord(53.125, -28.1, unit='deg', frame='icrs'), name='ECDFS'),
    FixedTarget(SkyCoord(352.360542, -0.844150, unit='deg', frame='icrs'), name='HSC-SSP_Deep2-3'),
    FixedTarget(SkyCoord(16.0, 4.0, unit='deg', frame='icrs'), name='Ecliptic 1'),
    FixedTarget(SkyCoord(30.0, 10.0, unit='deg', frame='icrs'), name='Ecliptic 2'),
    FixedTarget(SkyCoord(105.0, 20.0, unit='deg', frame='icrs'), name='Ecliptic 3'),
    FixedTarget(SkyCoord(6.022329,-72.081444, unit='deg', frame='icrs'), name='47 Tuc'),
    FixedTarget(SkyCoord(150.1, 2.18194444, unit='deg', frame='icrs'), name='COSMOS'),
    FixedTarget(SkyCoord(95.0, -25.0, unit='deg', frame='icrs'), name='Rubin_SV_95_-25'),
    #FixedTarget(SkyCoord(93.0, -20.0, unit='deg', frame='icrs'), name='Rubin_SV_95_-25'),
    FixedTarget(SkyCoord(125.0, -15.0, unit='deg', frame='icrs'), name='Rubin_SV_125_-15'),
]

Example sky viewer:
https://www.legacysurvey.org/viewer?ra=16.0&dec=4.&layer=des-dr1&zoom=6

In [ ]:
ra_target = np.array([target.ra.deg for target in targets])
dec_target = np.array([target.dec.deg for target in targets])

In [ ]:
m_extinction = np.load('/home/b/bechtol/rubin-user/rubin_sim_data/maps/DustMaps/dust_nside_64.npz')

In [ ]:
def plotEcliptic(sp):
    lon = np.linspace(0, 360, 100)
    lat = np.tile(0., 100)
    coord = SkyCoord(lon, lat, unit='deg', frame='geocentricmeanecliptic')
    sp.plot(coord.icrs.ra.deg, coord.icrs.dec.deg, c='black', ls='--', lw=1)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
sp = skyproj.Skyproj(ax=ax)
sp.draw_hpxmap(m_extinction['ebvMap'], cmap='Blues', lat_range=(-90., 90.), vmax=0.2, alpha=0.5)
cbar = sp.draw_inset_colorbar(label='E(B-V)', bbox_to_anchor=(-0.05, -0.15, 1, 1), loc='upper right', ticks=(0., 0.1, 0.2), format='%.1f')
plotEcliptic(sp)
sp.scatter(ra_target, dec_target, c='red', s=10, zorder=10)
plt.show()

In [ ]:
print('%20s%10s'%('Target', 'E(B-V)'))
for target in targets:
    ebv = m_extinction['ebvMap'][hp.ang2pix(64, target.ra.deg, target.dec.deg, lonlat=True)]
    print('%20s%10.2f'%(target.name, ebv))

In [ ]:
m_stellar_density = np.load('/home/b/bechtol/rubin-user/rubin_sim_data/maps/StarMaps/starDensity_g_nside_64.npz')

In [ ]:
#plt.figure()
#plt.hist(m_stellar_density['starDensity'][:,20], bins=np.linspace(0, 20000, 101))

In [ ]:
selection = (m_stellar_density['starDensity'][:,20] > 0.)
median_stellar_density = np.median(m_stellar_density['starDensity'][:,20][selection])
#print(median_stellar_density)

In [ ]:
print('%20s%20s'%('Target', 'Stellar Density'))
for target in targets:
    stellar_density = m_stellar_density['starDensity'][:,20][hp.ang2pix(64, target.ra.deg, target.dec.deg, lonlat=True)] / median_stellar_density
    print('%20s%20.2f'%(target.name, stellar_density))

# Visibility

Plot the airmass as a function of time during the night for three different dates over the next month, moving forward in time.

In [ ]:
observer = Observer.at_site("LSST")

In [ ]:
time = Time('2024-11-17 03:00:00.000', format='iso')

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
time = Time('2024-12-01 03:00:00.000', format='iso')

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
time = Time('2024-12-16 03:00:00.000', format='iso')

plt.figure()
plt.clf()
ax = plot_airmass(targets, observer, time, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

# Moon

Check when the Moon gets too close the targets

In [ ]:
time = Time('2024-11-16 03:00:00.000', format='iso')

time_next_month = time + np.arange(0., 30., 1.,) * u.day

lunar_illumination_next_month = observer.moon_illumination(time_next_month)

In [ ]:
x_ticks = Time(np.floor(time_next_month.mjd), format='mjd')
x_tick_labels = np.array([_.iso.split()[0] for _ in x_ticks])

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.axhspan(0., 20., alpha=0.2, color='black')

for target in targets:
    lunar_separation_next_month = observer.moon_altaz(time_next_month).separation(observer.altaz(time_next_month, target.coord)).deg
    ax1.plot(time_next_month.mjd, lunar_separation_next_month, label=target.name)

ax1.set_ylabel('Lunar Separation (deg)')
ax1.set_xticks(x_ticks.mjd[::3], x_tick_labels[::3], rotation=45.)
ax1.set_ylim(0., 180.)

ax2.plot(time_next_month.mjd, lunar_illumination_next_month, c='black', ls='--')
ax2.set_ylabel('Lunar Illumination')

ax1.legend(loc='upper center')

fig.tight_layout()